In [1]:
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.250:7077") \
        .appName("AlmaLundbergSparkapplicationA3")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.cores.max", 4)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 21:53:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Loading the text file from HDFS
lines_en = spark_context.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.en")
num_lines_en = lines_en.count()
print(f"Total lines in English file: {num_lines_en}") 

lines_sv = spark_context.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.sv")
num_lines_sv = lines_sv.count()
print(f"Total lines in Swedish file: {num_lines_sv}")

Total lines in English file: 1862234


Total lines in Swedish file: 1862234


In [3]:
# Counting nmr of partitions in the English file
num_partitions_en = lines_en.getNumPartitions()
print(f"Number of partitions in English file: {num_partitions_en}")

# Counting nmr of partitions in the Swedish file
num_partitions_sv = lines_sv.getNumPartitions()
print(f"Number of partitions in Swedish file: {num_partitions_sv}")

Number of partitions in English file: 2
Number of partitions in Swedish file: 3


In [4]:
def preprocess_text(line):
    return line.lower().split(' ')

In [5]:
processed_lines_en = lines_en.map(preprocess_text)
processed_lines_sv = lines_sv.map(preprocess_text)

In [6]:
# Verify that the line counts still match after the pre-processing:

processed_count_en = processed_lines_en.count()
processed_count_sv = processed_lines_sv.count()
print(f"Processed line count in English: {processed_count_en}")
print(f"Processed line count in Swedish: {processed_count_sv}")

Processed line count in English: 1862234
Processed line count in Swedish: 1862234


In [82]:
# The line count still match after pre-processing

In [7]:
# Check 10 most frequent words in English and Swedish

words_en = processed_lines_en.flatMap(lambda line: line)
words_sv = processed_lines_sv.flatMap(lambda line: line)

word_counts_en = words_en.map(lambda word: (word, 1)).reduceByKey(add)
word_counts_sv = words_sv.map(lambda word: (word, 1)).reduceByKey(add)

top_10_words_en = word_counts_en.takeOrdered(10, key=lambda x: -x[1])
top_10_words_sv = word_counts_sv.takeOrdered(10, key=lambda x: -x[1])

print("\nTop 10 words in English:")
for word, count in top_10_words_en:
    print(f'{word}: {count}')
    
print("\nTop 10 words in Swedish:")
for word, count in top_10_words_sv:
    print(f'{word}: {count}')


Top 10 words in English:
the: 3498375
of: 1659758
to: 1539760
and: 1288401
in: 1085993
that: 797516
a: 773522
is: 758050
for: 534242
we: 522849

Top 10 words in Swedish:
att: 1706293
och: 1344830
i: 1050774
det: 924866
som: 913276
för: 908680
av: 738068
är: 694381
en: 620310
vi: 539797


In [84]:
# The results are reasonable, as the most frequent words are common words used to build up sentences (konjunctions, prepositions etc)
# In the Swedish translation for example, the words also match most of the top 10 most frequent words in the Swedish language like i, och, att, det, som, en, är, av, för 

In [8]:
# Question A4

# Assign a unique index to each line in both RDDs and swap so that the line number is the key
sv_swapped = processed_lines_sv.zipWithIndex().map(lambda x: (x[1], x[0]))
en_swapped = processed_lines_en.zipWithIndex().map(lambda x: (x[1], x[0]))

In [9]:
# See how it looks, so everything was done correctly
sv_swapped.take(10)

[(0, ['återupptagande', 'av', 'sessionen']),
 (1,
  ['jag',
   'förklarar',
   'europaparlamentets',
   'session',
   'återupptagen',
   'efter',
   'avbrottet',
   'den',
   '17',
   'december.',
   'jag',
   'vill',
   'på',
   'nytt',
   'önska',
   'er',
   'ett',
   'gott',
   'nytt',
   'år',
   'och',
   'jag',
   'hoppas',
   'att',
   'ni',
   'haft',
   'en',
   'trevlig',
   'semester.']),
 (2,
  ['som',
   'ni',
   'kunnat',
   'konstatera',
   'ägde',
   '"den',
   'stora',
   'år',
   '2000-buggen"',
   'aldrig',
   'rum.',
   'däremot',
   'har',
   'invånarna',
   'i',
   'ett',
   'antal',
   'av',
   'våra',
   'medlemsländer',
   'drabbats',
   'av',
   'naturkatastrofer',
   'som',
   'verkligen',
   'varit',
   'förskräckliga.']),
 (3,
  ['ni',
   'har',
   'begärt',
   'en',
   'debatt',
   'i',
   'ämnet',
   'under',
   'sammanträdesperiodens',
   'kommande',
   'dagar.']),
 (4,
  ['till',
   'dess',
   'vill',
   'jag',
   'att',
   'vi,',
   'som',
   'ett',
 

In [10]:
# See how it looks, so everything was done correctly
en_swapped.take(10)

[(0, ['resumption', 'of', 'the', 'session']),
 (1,
  ['i',
   'declare',
   'resumed',
   'the',
   'session',
   'of',
   'the',
   'european',
   'parliament',
   'adjourned',
   'on',
   'friday',
   '17',
   'december',
   '1999,',
   'and',
   'i',
   'would',
   'like',
   'once',
   'again',
   'to',
   'wish',
   'you',
   'a',
   'happy',
   'new',
   'year',
   'in',
   'the',
   'hope',
   'that',
   'you',
   'enjoyed',
   'a',
   'pleasant',
   'festive',
   'period.']),
 (2,
  ['although,',
   'as',
   'you',
   'will',
   'have',
   'seen,',
   'the',
   'dreaded',
   "'millennium",
   "bug'",
   'failed',
   'to',
   'materialise,',
   'still',
   'the',
   'people',
   'in',
   'a',
   'number',
   'of',
   'countries',
   'suffered',
   'a',
   'series',
   'of',
   'natural',
   'disasters',
   'that',
   'truly',
   'were',
   'dreadful.']),
 (3,
  ['you',
   'have',
   'requested',
   'a',
   'debate',
   'on',
   'this',
   'subject',
   'in',
   'the',
   'course

In [11]:
# Join the two RDDs together on the line number key
sv_en_joined_rdd = sv_swapped.join(en_swapped)

In [12]:
# See how it looks
sv_en_joined_rdd.take(10) 

[(869580,
  (['europeiska',
    'unionen',
    'måste',
    'organisera',
    'återsändandet',
    'av',
    'illegala',
    'invandrare',
    'på',
    'ett',
    'sätt',
    'som',
    'upprätthåller',
    'strängaste',
    'respekt',
    'för',
    'mänskliga',
    'rättigheter',
    'och',
    'mänsklig',
    'värdighet.'],
   ['the',
    'european',
    'union',
    'must',
    'organise',
    'the',
    'return',
    'of',
    'illegal',
    'immigrants',
    'in',
    'a',
    'way',
    'that',
    'maintains',
    'the',
    'strictest',
    'respect',
    'for',
    'human',
    'rights',
    'and',
    'human',
    'dignity.'])),
 (870940,
  (['vår',
    'framtid',
    'är',
    'beroende',
    'av',
    'vår',
    'förmåga',
    'att',
    'bekämpa',
    'terrorismen.'],
   ['our',
    'future',
    'depends',
    'on',
    'our',
    'ability',
    'to',
    'defeat',
    'terrorism.'])),
 (883860,
  (['i',
    'och',
    'med',
    'dagens',
    'debatt',
    'och',
    '

In [13]:
# Filter function for the joined RDD according to lab requirements
def filter(lines):
    #Filter to exclude empty/missing lines
    filtered_rdd = lines.filter(lambda x: len(x[1][0]) > 1 and len(x[1][1]) > 1)

    #Filter sentences with a small number (the threshold) of words
    threshold = 5
    filtered_rdd = filtered_rdd.filter(lambda x: len(x[1][0]) < threshold and len(x[1][1]) < threshold)

    # Filter pairs with the same number of words
    filtered_rdd = filtered_rdd.filter(lambda x: len(x[1][0]) == len(x[1][1]))
    return filtered_rdd

In [14]:
# Call on the filter function to filter the joined RDD
filtered_rdd = filter(sv_en_joined_rdd)

In [15]:
# See how it looks, if it was filtered properly
filtered_rdd.take(10)

[(142910,
  (['jag', 'förklarar', 'debatten', 'avslutad.'],
   ['the', 'debate', 'is', 'closed.'])),
 (37045,
  (['de', 'har', 'ansvaret', 'tillsammans.'],
   ['they', 'are', 'jointly', 'responsible.'])),
 (129940,
  (['jag', 'förklarar', 'debatten', 'avslutad.'],
   ['that', 'concludes', 'the', 'debate.'])),
 (225205, (['krisen', 'på', 'balkan.'], ['the', 'balkan', 'crisis.'])),
 (369620, (['tack,', 'herr', 'minister.'], ['thank', 'you,', 'minister.'])),
 (651850,
  (['men', 'europa', 'har', 'svarat.'],
   ['but', 'europe', 'has', 'responded.'])),
 (684985,
  (['det', 'är', 'deras', 'bidrag.'],
   ['that', 'is', 'their', 'contribution.'])),
 (708435,
  (['den', 'är', 'onödigt', 'komplicerad.'],
   ['it', 'is', 'unnecessarily', 'complicated.'])),
 (708975,
  (['varför', 'hålla', 'det', 'hemligt?'], ['why', 'keep', 'it', 'secret?'])),
 (749190, (['\xa0\xa0', '.'], ['\xa0\xa0', '.']))]

In [16]:
# Pair words in the order they appear in each sentence pair
word_pairs_rdd = filtered_rdd.map(lambda x: zip(x[1][0], x[1][1]))

In [17]:
# See how it looks
print(list(word_pairs_rdd.take(1)[0])) 

[('nej,', 'no,'), ('nej!)', 'no!)')]


In [18]:
# Count the frequency of each word-translation pair
words_rdd = word_pairs_rdd.flatMap(lambda line: line)
word_counts_rdd = words_rdd.map(lambda word: (word, 1))
word_freq_rdd = word_counts_rdd.reduceByKey(lambda a, b: a + b)

# Print 10 of the most frequently occurring pairs of words
top_pairs = word_freq_rdd.takeOrdered(10, key=lambda x: -x[1])
for (word, translation), count in top_pairs:
    print(f'{word} - {translation}: {count}')


avslutad. - closed.: 2534
är - is: 2380
jag - the: 1324
debatten - is: 1324
förklarar - debate: 1317
debatten - the: 1225
härmed - is: 1215
är - debate: 1187
(artikel - (rule: 893
det - that: 852


In [ ]:
# The translations are not parfect, and the word pairs doesnt match in translation. However, the first two makes sens, and the last two. 
# The other ones are not translations of each other, but it looks like one of the most frequent short sentences perhaps could be 
# "härmed förklarar jag debatten avslutad", which is a completely different word structure on english - why the word pairs are a bit "scrambled"
# That is my conclusion at least.

In [30]:
# Section B - Working with DataFrames and SQL

dataframe = spark_session.read.csv("hdfs://192.168.2.250:9000/parking-citations.csv", header = True, inferSchema=True)
dataframe.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21 00:00:00|    1251.0|    NULL|       NULL|            CA|         200304.0|NULL|HOND|        PA|   GY|

In [31]:
dataframe.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Issue time: double (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: double (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: double (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: double (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [21]:
# Row count of CSV file
print(f"Number of rows: {dataframe.count()}")

Number of rows: 13077724


In [22]:
# Number of partitions in the underlying RDD
print(f"Number of partitions: {dataframe.rdd.getNumPartitions()}")

Number of partitions: 16


In [33]:
# Drop the columns VIN, Latitude, and Longitude
dataframe = dataframe.drop("VIN", "Latitude", "Longitude")

In [34]:
from pyspark.sql.functions import col

# Add new column from Fine amount as float
dataframe = dataframe.withColumn("FineAmountFloat", col("Fine amount").cast("float"))

# Show the DataFrame to verify the new column
dataframe.show()


+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+---------------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|Agency Description|Color Description|Body Style Description|FineAmountFloat|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+---------------+
|   1103341116|2015-12-21 00:00:00|    1251.0|    NULL|       NULL|            CA|         200304.0|HOND|        PA|   GY|     13147 WELBY WAY|01521|   1.

In [35]:
# Check that the new column was added
dataframe.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Issue time: double (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: double (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: double (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: double (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: double (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)
 |-- FineAmountFloat: float (nullable = true)



In [ ]:
# Find the maximum fine amount
max_fine = dataframe.agg({"FineAmountFloat": "max"}).collect()[0][0]

# Print the maximum fine amount
print(f"Maximum fine amount: {max_fine}")

In [32]:
# Count how many fines have this maximum amount
max_fine_count = dataframe.filter(col("FineAmountFloat") == max_fine).count()

print(f"Count of maximum fines: {max_fine_count}")

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `FineAmountFloat` cannot be resolved. Did you mean one of the following? [`Fine amount`, `Color`, `Location`, `Route`, `Agency`].;
'Filter ('FineAmountFloat = 1100.0)
+- Relation [Ticket number#1147,Issue Date#1148,Issue time#1149,Meter Id#1150,Marked Time#1151,RP State Plate#1152,Plate Expiry Date#1153,VIN#1154,Make#1155,Body Style#1156,Color#1157,Location#1158,Route#1159,Agency#1160,Violation code#1161,Violation Description#1162,Fine amount#1163,Latitude#1164,Longitude#1165,Agency Description#1166,Color Description#1167,Body Style Description#1168] csv


In [14]:
from pyspark.sql import functions as pyfuncs

# Show the top 20 most frequent vehicle makes, and their frequencies
vehicle_makes = dataframe.groupBy("Make").count()

sorted_makes = vehicle_makes.orderBy("Count", ascending=False).show(20)

24/03/06 21:34:55 INFO FileSourceStrategy: Pushed Filters: 
24/03/06 21:34:55 INFO FileSourceStrategy: Post-Scan Filters: 
24/03/06 21:34:55 INFO MemoryStore: Block broadcast_27 stored as values in memory (estimated size 200.7 KiB, free 433.4 MiB)
24/03/06 21:34:55 INFO MemoryStore: Block broadcast_27_piece0 stored as bytes in memory (estimated size 34.8 KiB, free 433.4 MiB)
24/03/06 21:34:55 INFO BlockManagerInfo: Added broadcast_27_piece0 in memory on host-192-168-2-147-de1:10005 (size: 34.8 KiB, free: 434.2 MiB)
24/03/06 21:34:55 INFO SparkContext: Created broadcast 27 from showString at NativeMethodAccessorImpl.java:0
24/03/06 21:34:55 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/03/06 21:34:55 INFO DAGScheduler: Registering RDD 71 (showString at NativeMethodAccessorImpl.java:0) as input to shuffle 6
24/03/06 21:34:55 INFO DAGScheduler: Got map stage job 16 (showString at NativeMethodAccess

+----+-------+
|Make|  count|
+----+-------+
|TOYT|2150768|
|HOND|1479996|
|FORD|1116235|
|NISS| 945133|
|CHEV| 892676|
| BMW| 603092|
|MERZ| 543298|
|VOLK| 432030|
|HYUN| 404917|
|DODG| 391686|
|LEXS| 368420|
| KIA| 328155|
|JEEP| 316300|
|AUDI| 255395|
|MAZD| 242344|
|OTHR| 205546|
| GMC| 184889|
|INFI| 174315|
|CHRY| 159948|
|SUBA| 154640|
+----+-------+
only showing top 20 rows



24/03/06 21:35:02 INFO TaskSetManager: Finished task 13.0 in stage 22.0 (TID 134) in 1729 ms on 192.168.2.250 (executor 1) (16/16)
24/03/06 21:35:02 INFO TaskSchedulerImpl: Removed TaskSet 22.0, whose tasks have all completed, from pool 
24/03/06 21:35:02 INFO DAGScheduler: ShuffleMapStage 22 (showString at NativeMethodAccessorImpl.java:0) finished in 7.273 s
24/03/06 21:35:02 INFO DAGScheduler: looking for newly runnable stages
24/03/06 21:35:02 INFO DAGScheduler: running: Set()
24/03/06 21:35:02 INFO DAGScheduler: waiting: Set()
24/03/06 21:35:02 INFO DAGScheduler: failed: Set()
24/03/06 21:35:02 INFO ShufflePartitionsUtil: For shuffle(6), advisory target size: 67108864, actual target size 1048576, minimum partition size: 1048576
24/03/06 21:35:02 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of codegened fast hashmap does not support this aggregate.
24/03/06 21:35:02 INFO SparkContext: Starting job: showString at NativeM

In [18]:
# Expand abbreviations in the color column using a User Defined Function
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define a dictionary mapping color abbreviations to their full names
COLORS = {
    'AL': 'Aluminum', 'AM': 'Amber', 'BG': 'Beige', 'BK': 'Black',
    'BL': 'Blue', 'BN': 'Brown', 'BR': 'Brown', 'BZ': 'Bronze',
    'CH': 'Charcoal', 'DK': 'Dark', 'GD': 'Gold', 'GO': 'Gold',
    'GN': 'Green', 'GY': 'Gray', 'GT': 'Granite', 'IV': 'Ivory',
    'LT': 'Light', 'OL': 'Olive', 'OR': 'Orange', 'MR': 'Maroon',
    'PK': 'Pink', 'RD': 'Red', 'RE': 'Red', 'SI': 'Silver', 'SL': 'Silver',
    'SM': 'Smoke', 'TN': 'Tan', 'VT': 'Violet', 'WT': 'White', 'WH': 'White',
    'YL': 'Yellow', 'YE': 'Yellow', 'UN': 'Unknown'
}

# The function takes a color abbreviation and returns the full color name using the dictionary defined above
# if there is no key matching the original color, the original color is returned
def expand_color_abbreviation(color_abbr):
    return COLORS.get(color_abbr, color_abbr)

# chekc the udf with Spark, specifying the return type as StringType
expand_color_udf = udf(expand_color_abbreviation, StringType())

# apply the udf to the Color column of the DataFrame, creates a new column ColorLong with expanded color names
dataframe = dataframe.withColumn("ColorLong", expand_color_udf(dataframe["Color"]))

In [19]:
# Find the most frequent color value for Toyotas (TOYT)
dataframe.filter(dataframe["Make"] == "TOYT").groupBy("ColorLong").count().orderBy("count", ascending=False).show(1)

24/03/06 21:43:08 INFO FileSourceStrategy: Pushed Filters: IsNotNull(Make),EqualTo(Make,TOYT)
24/03/06 21:43:08 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(Make#25),(Make#25 = TOYT)
24/03/06 21:43:09 INFO MemoryStore: Block broadcast_33 stored as values in memory (estimated size 200.7 KiB, free 432.7 MiB)
24/03/06 21:43:09 INFO MemoryStore: Block broadcast_33_piece0 stored as bytes in memory (estimated size 34.8 KiB, free 432.6 MiB)
24/03/06 21:43:09 INFO BlockManagerInfo: Added broadcast_33_piece0 in memory on host-192-168-2-147-de1:10005 (size: 34.8 KiB, free: 434.1 MiB)
24/03/06 21:43:09 INFO SparkContext: Created broadcast 33 from showString at NativeMethodAccessorImpl.java:0
24/03/06 21:43:09 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/03/06 21:43:09 INFO DAGScheduler: Registering RDD 93 (showString at NativeMethodAccessorImpl.java:0) as input to shuffle 8
24/03/06 21:43:09 INFO

+---------+------+
|ColorLong| count|
+---------+------+
|     Gray|489697|
+---------+------+
only showing top 1 row



24/03/06 21:43:16 INFO TaskSetManager: Finished task 14.0 in stage 28.0 (TID 169) in 1788 ms on 192.168.2.250 (executor 1) (16/16)
24/03/06 21:43:16 INFO TaskSchedulerImpl: Removed TaskSet 28.0, whose tasks have all completed, from pool 
24/03/06 21:43:16 INFO DAGScheduler: ShuffleMapStage 28 (showString at NativeMethodAccessorImpl.java:0) finished in 7.638 s
24/03/06 21:43:16 INFO DAGScheduler: looking for newly runnable stages
24/03/06 21:43:16 INFO DAGScheduler: running: Set()
24/03/06 21:43:16 INFO DAGScheduler: waiting: Set()
24/03/06 21:43:16 INFO DAGScheduler: failed: Set()
24/03/06 21:43:16 INFO ShufflePartitionsUtil: For shuffle(8), advisory target size: 67108864, actual target size 1048576, minimum partition size: 1048576
24/03/06 21:43:16 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of codegened fast hashmap does not support this aggregate.
24/03/06 21:43:16 INFO SparkContext: Starting job: showString at NativeM